# Downloading the data from crunchbase

In [1]:
import os
import requests
crunchbasefile = 'cb.tar.gz'
if not os.path.exists(cb_file):
    url = 'https://api.crunchbase.com/odm/v4/odm.tar.gz?user_key=' + os.environ.get('cb_api_key')
    response = requests.get(url)
    open(cb_file, 'wb').write(response.content)

In [ ]:
!tar -xvf cb.tar.gz --directory data

In [ ]:
# verify the checksum 
!md5sum data/organizations.csv

In [ ]:
# delete un-needed files
!rm data/checksum.csv data/people.csv

In [5]:
!dir

 Volume in drive D is New Volume
 Volume Serial Number is 2827-F736

 Directory of D:\MTech\Karya Consultant Assignment

01/28/2024  11:03 AM    <DIR>          .
01/28/2024  11:03 AM    <DIR>          ..
01/28/2024  10:53 AM    <DIR>          .ipynb_checkpoints
05/14/2019  05:22 AM            29,641 acquisitions.csv
05/14/2019  05:24 AM           450,560 bulk_export_sample.tar
01/28/2024  11:02 AM           154,033 bulk_export_sample.tar.gz
05/14/2019  05:22 AM             9,545 category_groups.csv
01/27/2024  06:58 PM             5,053 companies.xlsx
01/04/2020  07:30 PM    <DIR>          crunchbase-scraper-master
01/28/2024  12:19 AM            92,368 crunchbase-scraper-master.zip
01/28/2024  11:00 AM             2,105 Crunchbase.ipynb
05/14/2019  05:22 AM            15,189 degrees.csv
01/27/2024  10:50 PM                 0 error.csv
01/27/2024  10:49 PM                 0 error.txt
05/14/2019  05:22 AM            63,033 events.csv
05/14/2019  05:22 AM            22,844 event_appearan

In [88]:
import pandas as pd

organizations = pd.read_csv('organizations.csv')
organization_descriptions = pd.read_csv('organization_descriptions.csv')
result = pd.read_csv('result.csv')


In [89]:
# change the values of the name column to lowercase

result['Name'] = result['Name'].str.lower()
organizations['name'] = organizations['name'].str.lower()
organization_descriptions['name'] = organization_descriptions['name'].str.lower()

In [94]:
def find_competitors(company_name, industry_groups):
    competitors = organizations[
        organizations["category_groups_list"].str.split(r'\s*,\s*|\s*and\s*').apply(
            lambda x: any(item in x if isinstance(x, list) else False for item in industry_groups)
        )
    ]["name"].tolist()
    #competitors.remove(company_name)  # Remove the company itself from competitors
    return ", ".join(competitors)

# Apply the function to each row in result_df using the correct column names
result["Competitors"] = result.apply(
    lambda row: find_competitors(row["Name"], row["Industry"]),
    axis=1,
)


In [91]:
# population details of a company in result form the values in other tables

result["description"] = result["description"].combine_first(organization_descriptions["description"])
result["Sector"] = result["Sector"].combine_first(organizations["roles"])
result["Industry"] = result["Industry"].combine_first(organizations["category_list"])
result["Funding in USD"] = result["Funding in USD"].combine_first(organizations["total_funding_usd"])
result["Product"] = result["Product"].combine_first(organizations["category_groups_list"])
result["Competitors"] = result.apply(
    lambda row: find_competitors(row["Name"], row["Industry"]),
    axis=1,
)

In [95]:
result

,Name,Domain,description,Sector,Industry,Funding in USD,Product,Competitors
0,meridian entertainment group,http://meridianconcerts.com,The Corporate bonds Series OLI-III-B is offere...,company,"Consulting,Events",NaN,"Events,Media and Entertainment",
1,applied biocode,http://apbiocode.com,Padoq is a community network platform revoluti...,company,"Communities,Internet",NaN,"Community and Lifestyle,Internet Services",
2,woko,http://www.woko.fr/,Enrich lives. That's what we're all about. It'...,school,"E-Learning,Education,Higher Education",NaN,"Education,Software",
3,paladina health,http://www.paladinahealth.com,Kaffeen mobile application is the first mobile...,company,"Health Care,Hospital,Medical,Personal Health",165000000.0,Health Care,


In [97]:
result.to_csv('final_result.csv',index=False)